In [4]:
import importlib, features
importlib.reload(features)


<module 'features' from 'C:\\exits-lab\\src\\features.py'>

In [1]:
# Setup path so we can import our helper modules
import sys, os
sys.path.append(os.path.abspath("../src"))

from datetime import datetime, timedelta, timezone
from data import init_mt5, shutdown_mt5, pull_bars_mt5, save_parquet

# 1) Connect to MT5
init_mt5()

# 2) Configure symbol / timeframe / dates
symbol = "EURUSD"        # <-- change if your broker uses a suffix
tf = "M1"
end_utc = datetime.now(timezone.utc)
start_utc = end_utc - timedelta(days=7)

# 3) Pull bars
df = pull_bars_mt5(symbol, tf, start_utc, end_utc)
display(df.head(3))
display(df.tail(3))
print("Rows:", len(df), "Range:", df.index.min(), "→", df.index.max())

# 4) Save to parquet
out_path = f"../data/{symbol.lower()}_{tf.lower()}.parquet"
save_parquet(df, out_path)
print("Saved:", out_path)

# 5) Disconnect
shutdown_mt5()


open     high      low    close  tick_volume  \
datetime                                                                     
2025-08-12 15:33:00+00:00  1.16725  1.16727  1.16719  1.16720           15   
2025-08-12 15:34:00+00:00  1.16719  1.16721  1.16719  1.16719           15   
2025-08-12 15:35:00+00:00  1.16720  1.16738  1.16720  1.16734           36   

                           real_volume  spread  
datetime                                        
2025-08-12 15:33:00+00:00            0      11  
2025-08-12 15:34:00+00:00            0      12  
2025-08-12 15:35:00+00:00            0      10

open     high      low    close  tick_volume  \
datetime                                                                     
2025-08-19 11:30:00+00:00  1.16697  1.16697  1.16671  1.16671           38   
2025-08-19 11:31:00+00:00  1.16672  1.16677  1.16669  1.16670           34   
2025-08-19 11:32:00+00:00  1.16671  1.16677  1.16664  1.16665           22   

                           real_volume  spread  
datetime                                        
2025-08-19 11:30:00+00:00            0       9  
2025-08-19 11:31:00+00:00            0      10  
2025-08-19 11:32:00+00:00            0       9

Rows: 6954 Range: 2025-08-12 15:33:00+00:00 → 2025-08-19 11:32:00+00:00
Saved: ../data/eurusd_m1.parquet


In [2]:
import pandas as pd

df = pd.read_parquet("../data/eurusd_m1.parquet")
print(df.shape)
print(df.dtypes)
display(df.head())



(6954, 7)
open           float64
high           float64
low            float64
close          float64
tick_volume     uint64
real_volume     uint64
spread           int32
dtype: object


open     high      low    close  tick_volume  \
datetime                                                                     
2025-08-12 15:33:00+00:00  1.16725  1.16727  1.16719  1.16720           15   
2025-08-12 15:34:00+00:00  1.16719  1.16721  1.16719  1.16719           15   
2025-08-12 15:35:00+00:00  1.16720  1.16738  1.16720  1.16734           36   
2025-08-12 15:36:00+00:00  1.16732  1.16744  1.16732  1.16739           16   
2025-08-12 15:37:00+00:00  1.16745  1.16749  1.16742  1.16746           18   

                           real_volume  spread  
datetime                                        
2025-08-12 15:33:00+00:00            0      11  
2025-08-12 15:34:00+00:00            0      12  
2025-08-12 15:35:00+00:00            0      10  
2025-08-12 15:36:00+00:00            0      10  
2025-08-12 15:37:00+00:00            0       9

In [5]:
import pandas as pd
from features import build_features

# reload the parquet we just saved (or use df from memory)
raw = pd.read_parquet("../data/eurusd_m1.parquet")

feat = build_features(raw, atr_n=14, comp_short=14, comp_long=42, pip_size=1e-4)
display(feat.head(3))
display(feat[['atr','spread_pips','atr_pips','spread_to_atr','atr_compression','session_label']].tail(3))

# Save features for downstream notebooks
feat_out = "../data/eurusd_m1_features.parquet"
feat.to_parquet(feat_out)
print("Saved features:", feat_out, "rows:", len(feat))


open     high      low    close  tick_volume  \
datetime                                                                     
2025-08-12 16:27:00+00:00  1.16755  1.16759  1.16750  1.16752           22   
2025-08-12 16:28:00+00:00  1.16752  1.16752  1.16750  1.16751            4   
2025-08-12 16:29:00+00:00  1.16751  1.16751  1.16737  1.16743           23   

                           real_volume  spread       atr  atr_compression  \
datetime                                                                    
2025-08-12 16:27:00+00:00            0       9  0.000069         0.742482   
2025-08-12 16:28:00+00:00            0      12  0.000068         0.736027   
2025-08-12 16:29:00+00:00            0       9  0.000071         0.775602   

                           spread_points  spread_pips  atr_pips  \
datetime                                                          
2025-08-12 16:27:00+00:00            9.0          0.9  0.692857   
2025-08-12 16:28:00+00:00           12.0          1.2  0.678571   
2025-08-12 16:29:00+00:00            9.0          0.9  0.707143   

                           spread_to_atr  session_london  session_overlap  \
datetime                                                                    
2025-08-12 16:27:00+00:00       1.298969           False            False   
2025-08-12 16:28:00+00:00       1.768421           False            False   
2025-08-12 16:29:00+00:00       1.272727           False            False   

                           session_ny  session_asia session_label  
datetime                                                           
2025-08-12 16:27:00+00:00        True         False            ny  
2025-08-12 16:28:00+00:00        True         False            ny  
2025-08-12 16:29:00+00:00        True         False            ny

atr  spread_pips  atr_pips  spread_to_atr  \
datetime                                                                    
2025-08-19 11:30:00+00:00  0.000179          0.9  1.785714       0.504000   
2025-08-19 11:31:00+00:00  0.000168          1.0  1.678571       0.595745   
2025-08-19 11:32:00+00:00  0.000168          0.9  1.678571       0.536170   

                           atr_compression session_label  
datetime                                                  
2025-08-19 11:30:00+00:00         0.848485         other  
2025-08-19 11:31:00+00:00         0.798285         other  
2025-08-19 11:32:00+00:00         0.798867         other

Saved features: ../data/eurusd_m1_features.parquet rows: 6900
